In [ ]:
%load_ext lab_black

In [ ]:
import datetime as dt

import mplfinance as mpf
import pandas as pd
import numpy as np
import pandera as pa
import matplotlib.pyplot as plt

import shared


logger = shared.get_logger()

In [ ]:
klines_btc_1m_df = shared.read_binance_klines()
# trades_btc_df = shared.read_binance_trades()
klines_btc_15m_df = shared.split_df(df=klines_btc_1m_df, interval_str="15m")

### implement all prev highs track

In [ ]:
slice_df = klines_btc_15m_df.iloc[0:50]
highs_vals = []
highs_alines = []

for _, row in slice_df.iterrows():
    while highs_vals and highs_vals[-1] < row.high:
        highs_vals.pop()
        highs_alines.pop()
    ts = dt.datetime.fromtimestamp(row.open_time / 1000)
    aline = [(ts, row.high + 50), (ts, row.high + 100)]
    highs_vals.append(row.high)
    highs_alines.append(aline)
    assert len(highs_vals) == len(highs_alines)

mpf.plot(
    slice_df,
    warn_too_much_data=len(slice_df),
    alines={"alines": list(highs_alines), "colors": ["#f19d38"]},
    type="candle",
    figsize=(14, 4),
    style=shared.s,
)

### draw buy on every price intersection

In [ ]:
slice_df = klines_btc_15m_df.iloc[0:50]
highs_vals = []
buys_alines = []


for _, row in slice_df.iterrows():
    highs_poped = []
    while highs_vals and highs_vals[-1] < row.high:
        highs_poped.append(highs_vals.pop())
    highs_vals.append(row.high)
    if not highs_poped:
        continue
    end_ts = dt.datetime.fromtimestamp(row.open_time / 1000)
    start_ts = end_ts - dt.timedelta(minutes=15)
    aline = [(start_ts, highs_poped[0]), (end_ts, highs_poped[0])]
    buys_alines.append(aline)


mpf.plot(
    slice_df,
    warn_too_much_data=len(slice_df),
    alines={"alines": list(buys_alines), "colors": ["#f19d38"]},
    type="candle",
    figsize=(14, 4),
    style=shared.s,
)

### compare klines and trades

In [ ]:
from binance_historical_data import BinanceDataDumper


data_dumper = BinanceDataDumper(
    path_dir_where_to_dump="/home/jovyan/.var/binance",
    asset_class="spot",
    data_type="trades",
    data_frequency="1m",
)
data_dumper.dump_data(
    tickers=['BTCUSDT'],
    date_start=dt.date(year=2024, month=1, day=1),
    date_end=dt.date(year=2024, month=2, day=1),
    is_to_update_existing=True,
)

In [ ]:
file_path = '/home/jovyan/.var/binance/spot/monthly/trades/BTCUSDT/BTCUSDT-trades-2024-01.csv'
columns = ['id', 'price', 'qty', 'base_qty', 'time', 'is_buyer', 'is_maker']
trades_btc_df = (
    pd.read_csv(file_path, names=columns, header=None, nrows=1_000_000)
    .assign(
        _open_time=lambda x: x.time.apply(lambda x: dt.datetime.fromtimestamp(x/1000)),
        open=lambda x: x.price,
        close=lambda x: x.price,
        low=lambda x: x.price,
        high=lambda x: x.price,
    )
    .set_index('_open_time')
)

In [ ]:
end_ts = klines_btc_1m_df.index.min() + dt.timedelta(hours=5)
slice_klines_df = klines_btc_1m_df[lambda x: x.index <= end_ts]
# mpf.plot(slice_klines_df, type='candle', figsize=(14, 4), style=shared.s)
slice_trades_df = trades_btc_df[lambda x: x.index <= slice_klines_df.index.max()]
# mpf.plot( slice_trades_df, type='line', figsize=(14, 4), style=shared.s)
plt.style.use('dark_background')
plt.figure(figsize=(140,40))
plt.plot(slice_klines_df.index, slice_klines_df.open, 'r')
plt.plot(slice_trades_df.index, slice_trades_df.open, 'g--', alpha=0.5)
plt.show()

### implement buy on non klines data

In [ ]:
klines_from_trades_btc_1m_df = (
    trades_btc_df
    .sort_values(by=['time'])
    .assign(
        i=lambda x: x.time - x.time.min(),
        j=lambda x: (x.i / 60_000).astype(int),
    )
    .groupby(by=['j'])
    .agg(
        open_time=('time', 'min'),
        open=('price', 'first'),
        close=('price', 'last'),
        low=('price', 'min'),
        high=('price', 'max'),
    )
    .assign(
        date=lambda x: (x.open_time/1000).apply(dt.datetime.fromtimestamp),
    )
    .set_index('date')
)

In [ ]:
klines_btc_1m_df.head()[['open_time', 'open', 'close', 'low', 'high']]

In [ ]:
klines_from_trades_btc_1m_df.head()

In [ ]:
end_ts = klines_from_trades_btc_1m_df.index.min() + dt.timedelta(hours=5)
slice_klines_from_trades_df = klines_from_trades_btc_1m_df[lambda x: x.index <= end_ts-dt.timedelta(seconds=-1)]
slice_klines_df = klines_btc_1m_df[lambda x: x.index <= end_ts]
plt.style.use('dark_background')
plt.figure(figsize=(140,40))
plt.plot(slice_klines_from_trades_df.index, slice_klines_from_trades_df.open, 'r')
plt.plot(slice_klines_df.index, slice_klines_df.open, 'g--')
plt.show()

### implement this in tradingview with pinescript

```
// @version=5
indicator("scalp_2tops_2024_03_11", overlay=true)
var entersAmount = 0
var highsVals = array.new<float>(0)
buy = false
if time > timestamp("2024-03-08")
    highsPoped = array.new<float>(0)
    while highsVals.size() > 0
        if array.get(highsVals, highsVals.size()-1) < high
            highsPoped.push(highsVals.pop())
        else
            break
    array.push(highsVals, high)
    if highsPoped.size() > 0
        buy := true
        entersAmount += 1
plotshape(buy, style=shape.xcross)
if barstate.islast
    log.info("entereted trade[long] " + str.tostring(entersAmount) + " times")
```

### workout buys with spread

In [ ]:
slice_df = klines_btc_1m_df[
    lambda x: x.index >= dt.datetime.fromisoformat("2024-03-01")
][lambda x: x.index <= dt.datetime.fromisoformat("2024-03-07")]
highs_vals = []
buys_alines = []
spread_val = 300


for _, row in slice_df.iterrows():
    highs_poped = []
    while highs_vals and highs_vals[-1] < row.high:
        highs_poped.append(highs_vals.pop())
    highs_vals.append(row.high)
    if not highs_poped:
        continue
    if highs_vals[-1] > highs_poped[0] + spread_val:
        end_ts = dt.datetime.fromtimestamp(row.open_time / 1000)
        start_ts = end_ts - dt.timedelta(minutes=15)
        aline = [(start_ts, highs_poped[0]), (end_ts, highs_poped[0])]
        buys_alines.append(aline)


logger.info("buys_alines=%s", len(buys_alines))
mpf.plot(
    slice_df,
    warn_too_much_data=len(slice_df),
    alines={"alines": list(buys_alines), "colors": ["#f19d38"]},
    type="candle",
    figsize=(140, 40),
    style=shared.s,
)